import pandas as pd
import matplotlib.pyplot as plt
# ^^^ pyforest auto-imports - don't write above this line


# Main Question: Do American teams have more American Players than Canadian Teams and vice versa for 2018-2019? 

## Imports

In [1]:
import warnings
from functions import *
import pickle
sns.set(style="whitegrid")


Bad key "text.kerning_factor" on line 4 in
/Users/gregfeliu/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


## Last Year Available

In [3]:
last_year_query = f"""
    SELECT DISTINCT(season)
    FROM game
"""
last_year_query_result = run_query(last_year_query)

In [7]:
last_year = last_year_query_result[-1][0]

## Canadian Teams List

### All teams plus their ID

In [29]:
# not all of the franchises are in team_info
all_teams_query = f"""
    SELECT COUNT(*)
    FROM team_info
"""
all_teams_query_result = run_query(all_teams_query)

In [30]:
all_teams_query_result

[(17,)]

In [33]:
all_teams_query_result

[(17,)]

In [31]:
# from game_team_stats table
all_teams_query2 = f"""
    SELECT COUNT(DISTINCT(team_id))
    FROM game_teams_stats
    INNER JOIN game
        ON game.game_id = game_teams_stats.game_id
    WHERE game.season = {last_year}
""" # adding the last year clause removed 3 teams
all_teams_query2_result = run_query(all_teams_query2)

In [32]:
all_teams_query2_result

[(31,)]

### Canadian Team Ids 

In [35]:
# getting the actual team_id for each team
## also found here: https://www.kaggle.com/martinellis/nhl-game-data?select=team_info.csv
all_team_ids_query = f"""
    SELECT DISTINCT(team_id)
    FROM game_teams_stats
    INNER JOIN game
        ON game.game_id = game_teams_stats.game_id
    WHERE game.season = {last_year}
"""
all_team_ids_query_result = run_query(all_team_ids_query)

In [48]:
# flatten and sort
## https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
team_ids_flattened = [x for i in all_team_ids_query_result for x in i]
team_ids_sorted = sorted(team_ids_flattened)

In [49]:
canadian_team_dict = {'Vancouver': 26, 'Ottawa': 9,
                     'Montreal': 8, "Toronto": 10, "Calgary": 20,
                     "Winnipeg": 52, "Edmonton": 22}
canadian_team_ids = list(canadian_team_dict.values())

In [50]:
canadian_team_ids

[26, 9, 8, 10, 20, 52, 22]

### Values for "American" and "Canadian"

In [51]:
player_nat_query = f"""
    SELECT DISTINCT(nationality)
    FROM player_info
"""
player_nat_query_result = run_query(player_nat_query)

In [52]:
player_nat_query_result

[('nat',),
 ('RUS',),
 ('CAN',),
 ('SWE',),
 ('LTU',),
 ('USA',),
 ('CZE',),
 ('SVN',),
 ('FIN',),
 ('LVA',),
 ('DEU',),
 ('SVK',),
 ('DNK',),
 ('CHE',),
 ('BLR',),
 ('AUT',),
 ('FRA',),
 ('NLD',),
 ('NA',),
 ('UKR',),
 ('NGA',),
 ('HRV',),
 ('NOR',)]

### Test for a single team
only has players

In [53]:
last_ottawa_game_query = f"""
    SELECT game_id
    FROM game_teams_stats
    WHERE team_id = 9
    ORDER BY game_id DESC
    LIMIT 1
    
""" # when they were the home team
last_ottawa_game_query_result = run_query(last_ottawa_game_query)

In [55]:
last_ottawa_game_query_result

[(2018021260,)]

In [67]:
ottawa_nat_query = f"""
    SELECT DISTINCT(ps.player_id), ps.nationality
    FROM player_info ps
    INNER JOIN game_shifts gs
        ON ps.player_id = gs.player_id
    WHERE game_id = 2018021260
"""
ottawa_nat_query_result = run_query(ottawa_nat_query)

In [64]:
len(ottawa_nat_query_result)

19

In [70]:
all_nats = set(x[1] for x in ottawa_nat_query_result)

In [72]:
type(all_nats)

set

In [78]:
# getting the number of each nationality and their percentage 
ottawa_nat_dict = {x:0 for x in all_nats}
for x in ottawa_nat_query_result:
    ottawa_nat_dict[x[1]] += 1

In [82]:
num_players_on_team = len(ottawa_nat_query_result)
ottawa_nat_dict_pct = {key: round(value/num_players_on_team, 2) for key, value in ottawa_nat_dict.items()}

In [83]:
ottawa_nat_dict_pct

{'SWE': 0.11, 'USA': 0.32, 'FIN': 0.05, 'CAN': 0.47, 'CHE': 0.05}